In [93]:
import glob
import os.path

from lxml import etree
from lxml.etree import XMLSyntaxError

In [94]:
PATH: str = '../data/'
FILES: str = '*.xml'
NAMESPACE: dict = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [97]:
def extract_metadata(file_path: str = PATH + FILES) -> None:
    """Function for retrieving the needed metadata from the XML/TEI files."""
    correspondences: dict = {}
    try:
        for file in glob.glob(file_path):
            print(f"Working on {os.path.basename(file)}")
            tree = etree.parse(file)
            root = tree.getroot()

            sender: str
            receiver: str

            for elem in root.findall('./tei:teiHeader//tei:correspAction[@type="sent"]//tei:persName',
                                     NAMESPACE
                                     ):

                sender = elem.text
                print(f"Sender: {sender}")
            for elem in root.findall('./tei:teiHeader//tei:correspAction[@type="received"]//tei:persName',
                                     NAMESPACE
                                     ):
                receiver = elem.text
                print(f"Receiver: {receiver}")


    except XMLSyntaxError:
        print(f"There has been a syntax error in {file}.")
        pass


    print(correspondences)




In [98]:
extract_metadata()

Working on AWS-aw-05dx.xml
Sender: Louise Rosalie de Sinclair
Receiver: August Wilhelm von Schlegel
Working on AWS-aw-027u.xml
Sender: Johann Wolfgang von Goethe
Receiver: August Wilhelm von Schlegel
Working on AWS-aw-024n.xml
Sender: Friedrich Schleiermacher
Receiver: August Wilhelm von Schlegel
Working on AWS-aw-0512.xml
Sender: August Wilhelm von Schlegel
Receiver: Auguste Luise Adolfine von Flotow
Working on AWS-aw-02ae.xml
Sender: Jonathan Birch
Receiver: August Wilhelm von Schlegel
Working on AWS-aw-051s.xml
Sender: August Wilhelm von Schlegel
Receiver: Auguste Luise Adolfine von Flotow
Working on AWS-aw-00ya.xml
Sender: Ludwig Tieck
Receiver: August Wilhelm von Schlegel
Working on AWS-aw-0506.xml
Sender: August Wilhelm von Schlegel
Receiver: Karl B. Hase
Working on AWS-aw-053l.xml
Sender: August Wilhelm von Schlegel
Receiver: Unbekannt
Working on AWS-aw-050w.xml
Sender: August Wilhelm von Schlegel
Receiver: Auguste Luise Adolfine von Flotow
Working on AWS-aw-00xe.xml
Sender: Lud